# Project Zeus
## Created by Michael Wentz

### Import Modules

In [ ]:
import numpy as np
import pandas as pd
import math

from bokeh.io import show, output_notebook, push_notebook
from bokeh.layouts import widgetbox
from bokeh.models import LinearAxis, Range1d, DataRange1d, NumeralTickFormatter
from bokeh.plotting import figure
from bokeh.models.widgets import Slider, RadioButtonGroup
from ipywidgets import interactive

### Load production data from excel file

In [ ]:
file = 'https://raw.githubusercontent.com/mwentzWW/petrolpy/master/Zeus/Sample_Production/Sample_Prod_Data.xlsx'
well_data = pd.read_excel(file)
well_data.columns = [c.lower() for c in well_data.columns]
well_data.info()

### Check Data Layout

In [ ]:
well_data.head()

### Delete API because there is only one well, and we don't need water

In [ ]:
well_data.drop(columns=['api number', 'water_bbl'], inplace=True)
well_data.head()

### Replace NaN with zeros for computations

In [ ]:
well_data.fillna(value=0, inplace=True)

### Normalize data by time

In [ ]:
num_months = len(well_data['oil_bbl'])
num_months

In [ ]:
months = []
for x in range(0, num_months):
    months.append(x)
months[0:10]

In [ ]:
well_data['time'] = months
print(well_data.head())

### Calculate the type curve for oil and gas, assume b = 0.5 (will adjust later), use first 30 day average decline for di

In [ ]:
print(well_data.head())

In [ ]:
print(well_data.loc[[3], ('gas_mcf')])

In [ ]:
def plot_oil(qi_oil= well_data.oil_bbl.max(), b_value=0.8, initial_decline_rate=15, start_of_exp=well_data.time.max(), exp_yes_no=False, exp_decline_rate=10):
    # Decline rates are input as annual whole numbers, then converted to decimal monthly numbers
    initial_decline_rate = (initial_decline_rate/100)/12
    exp_decline_rate = (exp_decline_rate/100)/12
    
    well_data['type_curve_oil'] = qi_oil*((1 + b_value*initial_decline_rate*well_data.time)**(-1/b_value))
    if exp_yes_no == True:
        start_of_exp = int(start_of_exp)
        well_data['type_curve_expon'] = well_data.loc[start_of_exp, ('type_curve_oil')]*2.71828**(-exp_decline_rate*(well_data.time-start_of_exp))
        well_data.loc[start_of_exp:, ('type_curve_oil')] = 0
        well_data.loc[0:start_of_exp-1, ('type_curve_expon')] = 0
        connector = well_data.loc[start_of_exp, ('type_curve_expon')]
        well_data.type_curve_oil = well_data.type_curve_oil + well_data.type_curve_expon
        well_data.type_curve_expon.astype('float')
    well_data.type_curve_oil.astype('float')
    well_data.fillna(value=0, inplace=True)
    well_data.loc[well_data.type_curve_oil > qi_oil] = qi_oil
    
    p = figure(title= 'Oil Type Curve', y_axis_type='log')
    p.line(well_data.time, well_data.oil_bbl, line_color= 'green', line_width= 2, legend= 'Oil Production')
    p.line(well_data.time, well_data.type_curve_oil, line_color= 'black', line_width= 2, line_dash= 'dashed', legend= 'Curve Fit')
    
    
    
    p.yaxis.axis_label = 'BOPM'
    p.xaxis.axis_label = 'MONTH'
    p.y_range.start = 10
    p.yaxis[0].formatter = NumeralTickFormatter(format="0,0")
    p.xaxis[0].formatter = NumeralTickFormatter(format="0,0")
    p.title.align = 'center'
    
    
    show(p, notebook_handle=True)


def plot_gas(qi_gas= 1000, b_value=0.8, initial_decline_rate=0.15):
    well_data['type_curve_gas'] = qi_gas*((1 + b_value*initial_decline_rate*well_data.time)**(-1/b_value))
    well_data.type_curve_gas.astype('int')
    well_data.fillna(value=0, inplace=True)
    well_data.loc[well_data.type_curve_gas > qi_gas] = qi_gas
    


### Plot production with type curves

In [ ]:
output_notebook()

In [ ]:
def update(b_value, initial_decline_rate, start_of_exp, exp_yes_no, exp_decline_rate):
    plot_oil(qi_oil= well_data.oil_bbl.max(), b_value=b_value, initial_decline_rate=initial_decline_rate, start_of_exp=start_of_exp, exp_yes_no=exp_yes_no, exp_decline_rate=exp_decline_rate)
    push_notebook()

In [ ]:
interactive_plot = interactive(update, b_value= (0, 2, 0.01), initial_decline_rate= (0, 150), exp_yes_no = False, start_of_exp= (1, well_data.time.max(), 1), exp_decline_rate=(0, 20));
output = interactive_plot.children[-1]
output.layout.height = '400px'
interactive_plot

In [ ]:
print(well_data)

## To-do

* add optimized solution by minimizing the error